### ACC (Ant Colony Correction) is a new idea for implementing Ant Colony Optimization

**Basic idea**: for a given "sufficiently good" path, try to use ACO to further optimize the path "piecewisely"

In [15]:
using DataFrames
using CSV
Cities = CSV.read("cities.csv")
Data = Array{Float64,2}([Cities[:X]'; Cities[:Y]']);

In [32]:
GoodPath = CSV.read("Good-Path-newest.csv");
GoodPath = Array{Int, 1}(GoodPath[:Path]);

In [4]:
using Primes

In [5]:
using LinearAlgebra ## for the function "norm"
using StatsBase ## for function "sample"

**Part I**: write the evaluation function "**Len_Eval**" (Length Evaluation)

In [114]:
function Len_Eval_1(GoodPath::Array{Int, 1}, Data::Array{Float64, 2})
    
    if sort(Array{Int,1}(collect(Set(GoodPath))))!=collect(0:size(Data,2)-1)
        error("The path misses some cities!!!")
    elseif GoodPath[1]!=0
        error("The path does not start from City 0!!!")
    elseif GoodPath[end]!=0
        error("The path does not end at City 0!!!")
    end
    
    Dist = 0.0
    for i=1:length(GoodPath)-1
        if ((i%10)==0)##&&(i!=1) ## i.e. step 11, 21, 31, 41, etc.
            if isprime(GoodPath[i]+1)
                Dist = Dist + norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
            else
                Dist = Dist +1.1*norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
            end
        else
            Dist = Dist + norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
        end
    end
    Dist
end

Len_Eval_1 (generic function with 1 method)

In [6]:
function Len_Eval_2(GoodPath::Array{Int, 1}, Data::Array{Float64, 2})
    
    if sort(Array{Int,1}(collect(Set(GoodPath))))!=collect(0:size(Data,2)-1)
        error("The path misses some cities!!!")
    elseif GoodPath[1]!=0
        error("The path does not start from City 0!!!")
    elseif GoodPath[end]!=0
        error("The path does not end at City 0!!!")
    end
    
    Dist = 0.0
    for i=1:length(GoodPath)-1
        if ((i%10)==1)&&(i!=1) ## i.e. step 11, 21, 31, 41, etc.
            if isprime(GoodPath[i]+1)
                Dist = Dist + norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
            else
                Dist = Dist +1.1*norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
            end
        else
            Dist = Dist + norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
        end
    end
    Dist
end

Len_Eval_2 (generic function with 1 method)

In [11]:
round(Int,Len_Eval_2(GoodPath, Data))

1532126

In [19]:
round(Int,Len_Eval_2(GoodPath, Data))

1532125

In [29]:
round(Int,Len_Eval_2(GoodPath, Data))

1532123

In [33]:
round(Int,Len_Eval_2(GoodPath, Data))

1532089

In [140]:
function Dummy_Eval(GoodPath::Array{Int, 1}, Data::Array{Float64, 2})
    Dist = 0.0        
    for i=1:length(GoodPath)-1
        Dist = Dist + norm(Data[:, GoodPath[i]+1]-Data[:, GoodPath[i+1]+1])
    end
    Dist
end

Dummy_Eval (generic function with 1 method)

In [132]:
L_eval_1 = Float64[]
for i=18:30
    GoodPath = CSV.read("History-Results//Z_$(i)_10.csv");
    GoodPath = Array{Int, 1}(GoodPath[:Path])
    push!(L_eval_1, round(Len_Eval_1(GoodPath, Data), RoundUp))
end
L_eval_1;

In [133]:
L_eval_2 = Float64[]
for i=18:30
    GoodPath = CSV.read("History-Results//Z_$(i)_10.csv");
    GoodPath = Array{Int, 1}(GoodPath[:Path])
    push!(L_eval_2, round(Len_Eval_2(GoodPath, Data), RoundUp))
end
L_eval_2;

In [143]:
L_dummy = Float64[]
for i=18:30
    GoodPath = CSV.read("History-Results//Z_$(i)_10.csv");
    GoodPath = Array{Int, 1}(GoodPath[:Path])
    push!(L_dummy, round(Dummy_Eval(GoodPath, Data), RoundUp))
end
L_dummy;

In [134]:
L = [1683265.02, 1675956.66, 1672222.78, 1668616.97, 1664058.79, 1663115.37, 1657609.28, 1658854.95, 1658440.87, 
1656901.38, 1661120.29, 1658562.90, 1661280.17];

In [144]:
using Statistics
(L_eval_1+L_eval_2)/2-L
[L_eval_1-L L_eval_2-L L_dummy-L]
[mean(L_eval_1-L) mean(L_eval_2-L) mean(L_dummy-L)]

1×3 Array{Float64,2}:
 -18.3408  -6.95615  -15024.5

**Conclusion**: We should take the second evaluation method. 

**Part II**: Implement ACC (Ant Colony Correction)

In [148]:
N = 999
Path = [0;sample(collect(1:N),N, replace = false); 0]
Data = rand(2,N+1);

In [99]:
"""
ACC (Ant Colony Correction) is a function implementing the idea of ACO (Ant Colony Optimization)
to further correct a given path to a better one.

"""
function ACC(Path::Array{Int,1}, Data::Array{Float64,2}, a::Int, k::Int
        ; alp = 1.0, bet = 1.0, rho = 0.3, Q = 1.0, N_a = 500)
    
    b = a+k-1
    Pts = Data[:, Path[a:b].+1]
    Dist = [norm(Pts[:, i]-Pts[:, j]) for i=1:k, j=1:k]

    Tau = 1/(k-1)*ones(k,k)-1/(k-1)*Matrix{Float64}(I,k,k); ## the prior transition probability
    path_a = collect(1:k) ## record the current path

    path = collect(1:k) ## record the best path
    L = Inf ## record the best length
    
    for i=1:N_a
        Cand = setdiff(collect(1:k),[1, k]) ## the candidates
        CP = 1 ## cp =  current point
        path_a = Int[copy(CP)]

        for p = 2:k-1
            w = aweights((Tau[CP,Cand].^alp).*((Dist[CP,Cand].^(-1)).^bet))
            CP = sample(Cand,w)
            Cand = setdiff(Cand,[CP])
            push!(path_a, CP)
        end
        push!(path_a, k)
        path_a

        L_a = 0.0 ## total length of path_a
        for j = a:b-1
            if ((j%10)==1)&&(j!=1) ## i.e. step 11, 21, 31, 41, etc.
                if isprime(Path[a:b][path_a[j-a+1]]+1)
                    L_a = L_a + Dist[path_a[j-a+1], path_a[j-a+2]]
                else
                    L_a = L_a +1.1*Dist[path_a[j-a+1], path_a[j-a+2]]
                end
            else
                L_a = L_a + Dist[path_a[j-a+1], path_a[j-a+2]]
            end
        end
        ## println(L_a)

        if L_a<L
            path = copy(path_a)
            L = copy(L_a)
        end

        Tau = Tau*(1-rho) ## evaporation update
        ## Pheromone update
        PheroPerUnit = Q/L_a

        for j = 1:k-1
            Tau[path_a[j], path_a[j+1]] = Tau[path_a[j], path_a[j+1]] + PheroPerUnit ## Pheromone update
        end

        Tau   
    end
    
    ## (new path, new length)
    (Path[a:b][path_a], L)
end

ACC

In [115]:
a = 3
k = 20

alp = 1.0
bet = 1.0
rho = 0.3
Q = 1.0

ACC_I = ACC(GoodPath, Data, a::Int, k::Int; alp = 1.0, bet = 1.0, rho = 0.3, Q = 1.0, N_a = 500)
P_ACC = ACC_I[1];

Path = GoodPath[a:a+k-1]

using Plots
plotly()
scatter(Data[1,Path.+1], Data[2, Path.+1])
plot!(Data[1,Path.+1], Data[2, Path.+1])

plot!(Data[1,P_ACC.+1], Data[2, P_ACC.+1])

In [20]:
"""
**RandPts** is a function that randomly uniformly generates m points in dimension size(Range,2) inside Range. <br>
Inputs: <br>
m: number of random points to generate <br>
Range: dim x 2 matrix; the d-th row is the range of the d-th coordinate <br>
Output: <br>
an dim x m matrix; each column is a desired random vector. <br>
"""
function RandPts(Range::Array{Float64,2}, m::Int)
    ## dim = size(Range,1)
    ## randPts = rand(dim,m)
    ## Coeff = (Range[:,2].-Range[:,1])
    ## Intercept = Range[:,1]
    ## randPts = ((Range[:,2].-Range[:,1]).*rand(size(Range,1),m)).+Range[:,1]
    return ((Range[:,2].-Range[:,1]).*rand(size(Range,1),m)).+Range[:,1]
end

RandPts

In [107]:
"""
**ParaSelectACO** is a function that selects appropriate parameters for ACO.
"""
function ParaSelectACC(Range::Array{Float64, 2}, N_sample::Int, Path::Array{Int,1}, 
        Data::Array{Float64,2}, a::Int, k::Int; N_a=200::Int)
    RP = RandPts(Range, N_sample)
    Best_Length = Inf
    Best_Path = Int[]
    Best_Para = zeros(4)
    ## History = Float64[]
    for i=1:N_sample
        AP = ACC(Path, Data, a, k; alp = RP[1], bet = RP[2], rho = RP[3], Q = RP[4], N_a = N_a)
        ## push!(History, AP[2])
        if AP[2]<Best_Length
            Best_Length = AP[2]
            Best_Path = AP[1]
            Best_Para = RP[:,i]
        end
    end
    
    b = a+k-1
    L_o = 0.0 ## total length of the original path
    for j = a:b-1
        if ((j%10)==1)&&(j!=1) ## i.e. step 11, 21, 31, 41, etc.
            if isprime(Path[a:b][j-a+1]+1)
                L_o = L_o + norm(Data[:, Path[a:b][j-a+1]+1]-Data[:, Path[a:b][j-a+2]+1])
            else
                L_o = L_o +1.1*norm(Data[:, Path[a:b][j-a+1]+1]-Data[:, Path[a:b][j-a+2]+1])
            end
        else
            L_o = L_o + norm(Data[:, Path[a:b][j-a+1]+1]-Data[:, Path[a:b][j-a+2]+1])
        end
    end
    L_o
    
    return Dict("Best_Length"=>Best_Length, "Old_length"=>L_o, "Best_Path"=>Best_Path, 
        "Length_History"=>History, "Best_Para"=>Best_Para)
end

ParaSelectACC

In [121]:
Range = [0.0 5.0; 0.0 5.0; 0.0 1.0; 0.0 30.0]
N_sample = 500;
a = 30
k = 20

@time PSACC = ParaSelectACC(Range, N_sample, GoodPath, Data, a, k; N_a = 500::Int)

 30.387326 seconds (95.14 M allocations: 9.279 GiB, 14.55% gc time)


Dict{String,Any} with 5 entries:
  "Best_Length"    => 184.444
  "Old_length"     => 138.054
  "Best_Path"      => [111804, 69414, 2187, 177800, 95592, 167893, 2832, 97649,…
  "Length_History" => [284.499, 290.777, 409.238, 308.275, 291.035, 305.321, 31…
  "Best_Para"      => [1.55821, 1.99644, 0.727417, 25.6211]

In [122]:
Path = GoodPath[a:a+k-1]

using Plots
plotly()
scatter(Data[1,Path.+1], Data[2, Path.+1])
plot!(Data[1,Path.+1], Data[2, Path.+1])

Path_a = PSACC["Best_Path"]
plot!(Data[1,Path_a.+1], Data[2, Path_a.+1])

In [118]:
include("src//ParaSelectACO.jl");

In [196]:
"1"*"2"

"12"

In [194]:
k = 20
Imp_total = 0.0 ## improve total

for i=1:100
    a = sample(collect(1:size(Data,2)-k+2))
    b = a+k-1
    
    ## Take out the point coordinates
    Pts = Data[:,GoodPath[a:a+k-1].+1]
    
    ## Compute the distance matrix
    D = [norm(Pts[:,i]-Pts[:,j]) for i=1:k, j=1:k]
    
    Path_o = collect(1:k)
    L_o = 0.0 ## total length of the old path
    for j = a:b-1
        if ((j%10)==1)&&(j!=1) ## i.e. step 11, 21, 31, 41, etc.
            if isprime(GoodPath[a:b][j-a+1]+1)
                L_o = L_o + D[j-a+1, j-a+2]
            else
                L_o = L_o +1.1*D[j-a+1, j-a+2]
            end
        else
            L_o = L_o + D[j-a+1, j-a+2]
        end
    end
    L_o

    ## 
    
    PSACO = ParaSelectACO(Range, N_sample, D::Array{Float64,2}, 1, k; Na=200::Int)
    Path_a = PSACO["Best_Path"]

    L_n = 0.0 ## total length of the new path
    for j = a:b-1
        if ((j%10)==1)&&(j!=1) ## i.e. step 11, 21, 31, 41, etc.
            if isprime(GoodPath[a:b][Path_a][j-a+1]+1)
                L_n = L_n + D[Path_a[j-a+1], Path_a[j-a+2]]
            else
                L_n = L_n +1.1*D[Path_a[j-a+1], Path_a[j-a+2]]
            end
        else
            L_n = L_n + D[Path_a[j-a+1], Path_a[j-a+2]]
        end
    end
    L_n
    
    if L_n<L_o
        Improve = L_o-L_n
        Imp_total = Imp_total+ Improve
        println("a = $a, Improve = $Improve")
        GoodPath[a:b] = GoodPath[a:b][Path_a]
    end
end
Imp_total

a = 154165, Improve = 0.6788705425304329
a = 46941, Improve = 0.48328432627440066
a = 84107, Improve = 0.02978932927516098
a = 102461, Improve = 0.4990763380200747
a = 75089, Improve = 0.18776227662792166
a = 139577, Improve = 0.42256854491404283
a = 120801, Improve = 1.2028237690235244
a = 180927, Improve = 0.5787086926163312
a = 81498, Improve = 0.6650530548060942


4.7479368740879835

In [193]:
Imp_total

5.436531031155823

In [209]:
include("Code-Execute-ACC-Good-Path-2.jl")

a = 93600, Improve = 0.27852558570971553
a = 104340, Improve = 0.03564385669949388
a = 86890, Improve = 0.07802321434368764
a = 50726, Improve = 0.38679254747377456
a = 143078, Improve = 0.801479014230452


"Good-Path-2_new.csv"

In [201]:
GoodPath_new = CSV.read("Good-Path-1_new.csv");
GoodPath_new = Array{Int, 1}(GoodPath_new[:Path]);

In [204]:
Len_Eval_2(GoodPath_new, Data)

1.5333973490822697e6

In [205]:
GoodPath = CSV.read("Good-Path-1.csv");
GoodPath = Array{Int, 1}(GoodPath[:Path]);

In [208]:
Len_Eval_2(GoodPath, Data)-Len_Eval_2(GoodPath_new, Data)

7.85535231535323

In [207]:
[Len_Eval_2(GoodPath_new, Data) Len_Eval_2(GoodPath, Data)]

1×2 Array{Float64,2}:
 1.5334e6  1.53341e6

In [ ]:
using Plots
plotly()
scatter(Pts[1,:], Pts[2,:])

plot!(Pts[1,:], Pts[2,:], label = "Old Path, length = $(round(L_o, RoundUp, digits = 2))")
plot!(Pts[1,Path_a], Pts[2,Path_a], label = "New Path, length = $(round(L_n, RoundUp, digits = 2))")
PSACO = ParaSelectACO(Range, N_sample, D::Array{Float64,2}, 1, k; Na=200::Int)
Path_a = PSACO["Best_Path"]

In [4]:
K = collect(18:30)
L = [1683265.02, 1675956.66, 1672222.78, 1668616.97, 1664058.79, 1663115.37, 1657609.28, 1658854.95, 1658440.87, 
1656901.38, 1661120.29, 1658562.90, 1661280.17];

In [6]:
using Plots
plotly()
plot(K,L)
scatter!(K,L)

In [7]:
minimum(L)*100/101

1.640496415841584e6